In [1]:
import neutron_analysis.shielding as shielding
import neutron_analysis.utils as utils

import pandas as pd
import numpy as np

# Shielding

In [2]:
from importlib import reload
reload(shielding)

<module 'neutron_analysis.shielding' from '/Users/wpk/Documents/Research/CurrentResearch/Projects/CCI/collaboration_646/Nick_sharp_workflow/neutron_analysis/neutron_analysis/shielding.py'>

In [3]:
df_mass = (
    shielding.read_mass_excel('./example_input_data.xlsx', sheet_name='Sheet1', extra_index=['sample','mass'])
)
df_mass.head()

mass_percent  mass_frac
sample   mass radius length density  element                         
sample_0 0.3  0.65   0.15   1.506792 H                0.75     0.0075
                                     C                2.00     0.0200
                                     N                1.00     0.0100
                                     O               47.00     0.4700
                                     Na               1.22     0.0122

In [4]:
# create shielding objects
S = shielding.NeutronShieldingUser(df_mass)

In [5]:
S.mass_table.head()

mass_percent  mass_frac
sample   mass radius length density  element                         
sample_0 0.3  0.65   0.15   1.506792 H                0.75     0.0075
                                     C                2.00     0.0200
                                     N                1.00     0.0100
                                     O               47.00     0.4700
                                     Na               1.22     0.0122

In [6]:
S.sigma_table.head()

mass_percent  mass_frac  \
sample   mass radius length density  element                            
sample_0 0.3  0.65   0.15   1.506792 H                0.75     0.0075   
                                     C                2.00     0.0200   
                                     N                1.00     0.0100   
                                     O               47.00     0.4700   
                                     Na               1.22     0.0122   

                                              atomic_mass  sigma_a  sigma_s  \
sample   mass radius length density  element                                  
sample_0 0.3  0.65   0.15   1.506792 H           1.007940  0.33260   82.020   
                                     C          12.011000  0.00350    5.551   
                                     N          14.006740  1.90000   11.510   
                                     O          15.999400  0.00019    4.232   
                                     Na         22.989768  0.53000    3.280   

                                               Sigma_a   Sigma_s  
sample   mass radius length density  element                      
sample_0 0.3  0.65   0.15   1.506792 H        0.002246  0.553784  
                                     C        0.000005  0.008387  
                                     N        0.001231  0.007456  
                                     O        0.000005  0.112806  
                                     Na       0.000255  0.001579

In [7]:
S.sigma_table_tot

,,,,,mass_frac,Sigma_a,Sigma_s,Sigma_t
sample,mass,radius,length,density,,,,
sample_0,0.3,0.65,0.15,1.506792,0.99876,0.010283,0.718763,0.729046
sample_1,0.4,0.65,0.15,2.009056,0.99876,0.013711,0.958350,0.972061
sample_2,0.3,0.65,0.15,1.506792,1.00126,0.011032,0.903357,0.914389


In [8]:
S.shielding_table

,,,,,f_scatter,f0_noscatter
sample,mass,radius,length,density,,
sample_0,0.3,0.65,0.15,1.506792,0.997731,0.995817
sample_1,0.4,0.65,0.15,2.009056,0.997114,0.994619
sample_2,0.3,0.65,0.15,1.506792,0.997653,0.995551


In [10]:
S.shielding_ratio('sample_0')

sample_0/sample                 uncert  \
                                           f_scatter f0_noscatter f_scatter   
sample   mass radius length density                                           
sample_0 0.3  0.65   0.15   1.506792        1.000000     1.000000  0.000000   
sample_1 0.4  0.65   0.15   2.009056        1.000619     1.001205  0.000126   
sample_2 0.3  0.65   0.15   1.506792        1.000077     1.000267  0.000016   

                                                   
                                     f0_noscatter  
sample   mass radius length density                
sample_0 0.3  0.65   0.15   1.506792     0.000000  
sample_1 0.4  0.65   0.15   2.009056     0.000245  
sample_2 0.3  0.65   0.15   1.506792     0.000054

In [13]:
S.shielding_ratio('sample_0', value_name='values', exclude_std=False)

values              sample_0/sample  \
                                     f_scatter f0_noscatter       f_scatter   
sample   mass radius length density                                           
sample_0 0.3  0.65   0.15   1.506792  0.997731     0.995817        1.000000   
sample_1 0.4  0.65   0.15   2.009056  0.997114     0.994619        1.000619   
sample_2 0.3  0.65   0.15   1.506792  0.997653     0.995551        1.000077   

                                                     uncert               
                                     f0_noscatter f_scatter f0_noscatter  
sample   mass radius length density                                       
sample_0 0.3  0.65   0.15   1.506792     1.000000  0.000000     0.000000  
sample_1 0.4  0.65   0.15   2.009056     1.001205  0.000126     0.000245  
sample_2 0.3  0.65   0.15   1.506792     1.000267  0.000016     0.000054

In [14]:
# normalized values by hand

In [15]:
std = S.shielding_table.query('sample=="sample_0"').iloc[0]
ratio = std / S.shielding_table
ratio

,,,,,f_scatter,f0_noscatter
sample,mass,radius,length,density,,
sample_0,0.3,0.65,0.15,1.506792,1.000000,1.000000
sample_1,0.4,0.65,0.15,2.009056,1.000619,1.001205
sample_2,0.3,0.65,0.15,1.506792,1.000077,1.000267


In [16]:
# uncert
uncert = np.abs(std - S.shielding_table) / 2 / np.sqrt(6)
uncert

,,,,,f_scatter,f0_noscatter
sample,mass,radius,length,density,,
sample_0,0.3,0.65,0.15,1.506792,0.000000,0.000000
sample_1,0.4,0.65,0.15,2.009056,0.000126,0.000245
sample_2,0.3,0.65,0.15,1.506792,0.000016,0.000054


# photon attenuation

In [17]:
import neutron_analysis.photonatten as photonatten
import pandas as pd

In [22]:
reload(shielding)
reload(photonatten)

<module 'neutron_analysis.photonatten' from '/Users/wpk/Documents/Research/CurrentResearch/Projects/CCI/collaboration_646/Nick_sharp_workflow/neutron_analysis/neutron_analysis/photonatten.py'>

In [23]:
df_mass = (
    shielding.read_mass_excel('./example_input_data.xlsx', sheet_name='mass', extra_index=['sample','mass'])
    
)

df_mass_limit = (
    df_mass
    .assign(track=lambda x: ~x['track'].isnull())
    .query('track')
    .drop('track', axis=1)
    
)

df_gamma = (
    pd.read_excel('./example_input_data.xlsx', 'gamma')
    .rename(columns={'element': 'element_tot'})
    .assign(element=lambda x: x['element_tot'].str.split('-').str.get(0))
    # energy should be in MeV
    # convert kev to mev
    .assign(energy=lambda x: x['gamma_kev'] / 1000.)
    # using mev in original sheed
    # NOTE: values of these are different
#     .assign(energy=lambda x: x['gamma_mev_used'])
    .set_index('element')

)

df_gamma_limit = (
    df_gamma
    .assign(energy=lambda x: x['gamma_mev_used'])
    
)


In [24]:
S = shielding.NeutronShieldingUser(df_mass)

p = photonatten.PhotonAtten(df_gamma, df_mass)
p_limit = photonatten.PhotonAtten(df_gamma_limit, df_mass_limit)

In [38]:
# atten frame
# this interpolates the tables from https://physics.nist.gov/PhysRefData/XrayMassCoef/tab3.html 
# which i scraped
photonatten.get_default_atten_table().query('element=="Al"').head()

,energy,mu_rho,muen_rho,element
438,0.00100,1185.0,1183.0,Al
439,0.00150,402.2,400.1,Al
440,0.00156,362.1,360.0,Al
441,0.00156,3957.0,3829.0,Al
442,0.00200,2263.0,2204.0,Al


In [39]:
# atten frame
# this interpolates the tables from https://physics.nist.gov/PhysRefData/XrayMassCoef/tab3.html 
# which i scraped
photonatten.get_default_atten_table().query('element=="Al"').head().energy.values

array([0.001     , 0.0015    , 0.00155959, 0.00155961, 0.002     ])

In [40]:
p.mass_table.head()

mass_frac track
sample mass     radius length density  element                 
std    0.107074 0.65   0.15   0.537795 As        0.000043   NaN
                                       Cd        0.000049   NaN
                                       W         0.000035   NaN
                                       Au        0.000009   NaN
                                       Sm        0.000007   NaN

In [41]:
p.gamma_table.head()

,element_tot,gamma_kev,gamma_mev_used,energy
element,,,,
Mg,Mg-27,843.00,0.843,0.84300
Al,Al-28,1778.80,1.778,1.77880
Ca,Ca-49,3084.50,3.084,3.08450
Cl,Cl-38,1642.59,1.643,1.64259
Mn,Mn-56,1810.70,1.817,1.81070


In [42]:
p.atten_table.head()

mass_frac  \
sample mass     radius length density  element energy element_other              
std    0.107074 0.65   0.15   0.537795 Mg      0.843  As              0.000043   
                                                      Au              0.000009   
                                                      C               0.444153   
                                                      Cd              0.000049   
                                                      H               0.062117   

                                                                       mu_rho  \
sample mass     radius length density  element energy element_other             
std    0.107074 0.65   0.15   0.537795 Mg      0.843  As             0.062884   
                                                      Au             0.082483   
                                                      C              0.069223   
                                                      Cd             0.064885   
                                                      H              0.137447   

                                                                     muen_rho  
sample mass     radius length density  element energy element_other            
std    0.107074 0.65   0.15   0.537795 Mg      0.843  As             0.025992  
                                                      Au             0.042331  
                                                      C              0.028650  
                                                      Cd             0.027987  
                                                      H              0.056997

In [43]:
p.atten_table_tot.head()

atten_coef  \
sample mass     radius length density  element energy                
std    0.107074 0.65   0.15   0.537795 Mg      0.84300    0.005930   
                                       Al      1.77880    0.004085   
                                       Ca      3.08450    0.003019   
                                       Cl      1.64259    0.004256   
                                       Mn      1.81070    0.004045   

                                                        self_atten_coef  
sample mass     radius length density  element energy                    
std    0.107074 0.65   0.15   0.537795 Mg      0.84300         0.997041  
                                       Al      1.77880         0.997960  
                                       Ca      3.08450         0.998492  
                                       Cl      1.64259         0.997875  
                                       Mn      1.81070         0.997980

In [44]:
p.atten_ratio(exclude_std=True, value_name='values')

values  \
                                                   self_atten_coef   
sample mass radius length density  element energy                    
sample 0.32 0.65   0.15   1.607245 Mg      0.84300        0.989265   
                                   Al      1.77880        0.992565   
                                   Ca      3.08450        0.994361   
                                   Cl      1.64259        0.992268   
                                   Mn      1.81070        0.992635   
                                   Na      1.36855        0.991546   
                                   Ti      0.32000        0.983639   
                                   V       1.43400        0.991750   
                                   Sc      0.88960        0.989524   
                                   Tb      0.87980        0.989469   
                                   Sr      0.51450        0.986612   
                                   Co      1.33260        0.991434   
                                   Eu      1.40800        0.991669   
                                   Rb      1.07720        0.990457   
                                   Ba      0.49626        0.986413   
                                   Cs      0.79620        0.988998   
QC     0.30 0.65   0.15   1.506792 Mg      0.84300        0.992268   
                                   Al      1.77880        0.994653   
                                   Ca      3.08450        0.995973   
                                   Cl      1.64259        0.994436   
                                   Mn      1.81070        0.994703   
                                   Na      1.36855        0.993912   
                                   Ti      0.32000        0.988259   
                                   V       1.43400        0.994060   
                                   Sc      0.88960        0.992454   
                                   Tb      0.87980        0.992415   
                                   Sr      0.51450        0.990364   
                                   Co      1.33260        0.993831   
                                   Eu      1.40800        0.994002   
                                   Rb      1.07720        0.993126   
                                   Ba      0.49626        0.990222   
                                   Cs      0.79620        0.992076   

                                                        std/sample  \
                                                   self_atten_coef   
sample mass radius length density  element energy                    
sample 0.32 0.65   0.15   1.607245 Mg      0.84300        1.007860   
                                   Al      1.77880        1.005436   
                                   Ca      3.08450        1.004155   
                                   Cl      1.64259        1.005651   
                                   Mn      1.81070        1.005385   
                                   Na      1.36855        1.006177   
                                   Ti      0.32000        1.012098   
                                   V       1.43400        1.006027   
                                   Sc      0.88960        1.007668   
                                   Tb      0.87980        1.007709   
                                   Sr      0.51450        1.009836   
                                   Co      1.33260        1.006260   
                                   Eu      1.40800        1.006087   
                                   Rb      1.07720        1.006978   
                                   Ba      0.49626        1.009986   
                                   Cs      0.79620        1.008058   
QC     0.30 0.65   0.15   1.506792 Mg      0.84300        1.004810   
                                   Al      1.77880        1.003326   
                                   Ca      3.08450        1.002530   
                                   Cl      1.64259        1.003458   

## look at limited (compare to excel)

In [45]:
p_limit.atten_table.query('element=="Mg" and sample=="std"')

mass_frac  \
sample mass     radius length density  element energy element_other              
std    0.107074 0.65   0.15   0.537795 Mg      0.843  C               0.444153   
                                                      H               0.062117   
                                                      O               0.493039   

                                                                       mu_rho  \
sample mass     radius length density  element energy element_other             
std    0.107074 0.65   0.15   0.537795 Mg      0.843  C              0.069223   
                                                      H              0.137447   
                                                      O              0.069333   

                                                                     muen_rho  
sample mass     radius length density  element energy element_other            
std    0.107074 0.65   0.15   0.537795 Mg      0.843  C              0.028650  
                                                      H              0.056997  
                                                      O              0.028670

In [46]:
p_limit.atten_table_tot.query('element=="Mg"')

,,,,,,,atten_coef,self_atten_coef
sample,mass,radius,length,density,element,energy,,
std,0.107074,0.65,0.15,0.537795,Mg,0.843,0.005927,0.997043
sample,0.320000,0.65,0.15,1.607245,Mg,0.843,0.020351,0.989893
QC,0.300000,0.65,0.15,1.506792,Mg,0.843,0.015230,0.992423


In [47]:
p_limit.atten_ratio(exclude_std=True)

std/sample  \
                                                   self_atten_coef   
sample mass radius length density  element energy                    
sample 0.32 0.65   0.15   1.607245 Mg      0.84300        1.007222   
                                   Al      1.77800        1.004997   
                                   Ca      3.08400        1.003821   
                                   Cl      1.64300        1.005193   
                                   Mn      1.81700        1.004941   
                                   Na      1.36855        1.005677   
                                   Ti      0.32008        1.011112   
                                   V       1.43400        1.005539   
                                   Sc      0.88960        1.007047   
                                   Tb      0.87980        1.007084   
                                   Sr      0.51450        1.009038   
                                   Co      1.33260        1.005752   
                                   Eu      1.40800        1.005594   
                                   Rb      1.07720        1.006413   
                                   Ba      0.49626        1.009175   
                                   Cs      0.79620        1.007405   
QC     0.30 0.65   0.15   1.506792 Mg      0.84300        1.004654   
                                   Al      1.77800        1.003219   
                                   Ca      3.08400        1.002448   
                                   Cl      1.64300        1.003346   
                                   Mn      1.81700        1.003182   
                                   Na      1.36855        1.003659   
                                   Ti      0.32008        1.007128   
                                   V       1.43400        1.003570   
                                   Sc      0.88960        1.004542   
                                   Tb      0.87980        1.004565   
                                   Sr      0.51450        1.005816   
                                   Co      1.33260        1.003708   
                                   Eu      1.40800        1.003606   
                                   Rb      1.07720        1.004134   
                                   Ba      0.49626        1.005904   
                                   Cs      0.79620        1.004771   

                                                            uncert  
                                                   self_atten_coef  
sample mass radius length density  element energy                   
sample 0.32 0.65   0.15   1.607245 Mg      0.84300        0.001474  
                                   Al      1.77800        0.001020  
                                   Ca      3.08400        0.000780  
                                   Cl      1.64300        0.001060  
                                   Mn      1.81700        0.001008  
                                   Na      1.36855        0.001159  
                                   Ti      0.32008        0.002268  
                                   V       1.43400        0.001131  
                                   Sc      0.88960        0.001438  
                                   Tb      0.87980        0.001446  
                                   Sr      0.51450        0.001845  
                                   Co      1.33260        0.001174  
                                   Eu      1.40800        0.001142  
                                   Rb      1.07720        0.001309  
                                   Ba      0.49626        0.001873  
                                   Cs      0.79620        0.001511  
QC     0.30 0.65   0.15   1.506792 Mg      0.84300        0.000950  
                                   Al      1.77800        0.000657  
                                   Ca      3.08400        0.000500  
                                   Cl      1.64300        0.000683  
                  

In [48]:
# maual ratio
df = p_limit.atten_table_tot
df = utils.loc_levels(df, {'element': df_gamma.index,'sample':['std','sample','QC']})[['self_atten_coef']]
ref = df.query('sample=="std"').droplevel([x for x in df.index.names if x not in ['element']])

ratio = (
    (ref / df.query('sample != "std"'))
#    .assign(uncert=lambda x: np.abs(x['self_atten_coef'] - 1)/2/np.sqrt(6))
)

uncert = np.sqrt(ratio - 1) / 2 / np.sqrt(6)

pd.concat({'ratio': ratio, 'uncert': uncert}, axis=1).head()

ratio  \
                                                  self_atten_coef   
sample mass radius length density  element energy                   
sample 0.32 0.65   0.15   1.607245 Mg      0.843         1.007222   
                                   Al      1.778         1.004997   
                                   Ca      3.084         1.003821   
                                   Cl      1.643         1.005193   
                                   Mn      1.817         1.004941   

                                                           uncert  
                                                  self_atten_coef  
sample mass radius length density  element energy                  
sample 0.32 0.65   0.15   1.607245 Mg      0.843         0.017347  
                                   Al      1.778         0.014430  
                                   Ca      3.084         0.012618  
                                   Cl      1.643         0.014709  
                                   Mn      1.817         0.014348

# write to excel file

In [49]:
with pd.ExcelWriter('./tmp.xlsx') as f:
    S.mass_table.to_excel(f, sheet_name='mass')
    S.sigma_table.to_excel(f, sheet_name='sigma_table')
    S.sigma_table_tot.to_excel(f, sheet_name='sigma_table_tot')
#    S.shielding_table.to_excel(f, sheet_name='shielding')
    S.shielding_ratio(value_name='values').to_excel(f, sheet_name='shielding_ratio')
    
    p.gamma_table.to_excel(f, sheet_name='gamma')
    p.atten_table.to_excel(f, sheet_name='photon_atten')
#     p.atten_table_tot.to_excel(f, sheet_name='photon_atten_tot')
    p.atten_ratio(value_name='values').to_excel(f, sheet_name='photon_ratio')



In [51]:
!open ./tmp.xlsx

In [145]:
!open tmp.xlsx

# some other stuff

In [103]:
out = (
    utils.ratio_table(p.atten_table_tot, ['sample','element'], 'self_atten_coef')
    .assign(uncert=lambda x: np.abs(x['self_atten_coef'] -1 ) /2 / np.sqrt(6))
)
out.unstack('sample').loc[df_gamma.index]

self_atten_coef                   uncert               
sample               QC    sample  std        QC    sample  std
element                                                        
Mg             1.004810  1.007860  1.0  0.000982  0.001604  0.0
Al             1.003326  1.005436  1.0  0.000679  0.001110  0.0
Ca             1.002530  1.004155  1.0  0.000516  0.000848  0.0
Cl             1.003458  1.005651  1.0  0.000706  0.001154  0.0
Mn             1.003295  1.005385  1.0  0.000673  0.001099  0.0
Na             1.003782  1.006177  1.0  0.000772  0.001261  0.0
Ti             1.007366  1.012098  1.0  0.001504  0.002469  0.0
V              1.003690  1.006027  1.0  0.000753  0.001230  0.0
Sc             1.004693  1.007668  1.0  0.000958  0.001565  0.0
Tb             1.004718  1.007709  1.0  0.000963  0.001574  0.0
Sr             1.006011  1.009836  1.0  0.001227  0.002008  0.0
Co             1.003832  1.006260  1.0  0.000782  0.001278  0.0
Eu             1.003726  1.006087  1.0  0.000761  0.001242  0.0
Rb             1.004272  1.006978  1.0  0.000872  0.001424  0.0
Ba             1.006101  1.009986  1.0  0.001245  0.002038  0.0
Cs             1.004931  1.008058  1.0  0.001006  0.001645  0.0

In [100]:
out = (
    utils.ratio_table(p_limit.atten_table_tot, ['sample','element'], 'self_atten_coef')
    .assign(uncert=lambda x: np.abs(x['self_atten_coef'] -1 ) /2 / np.sqrt(6))
)
out.unstack('sample').loc[df_gamma.index]

self_atten_coef                   uncert               
sample               QC    sample  std        QC    sample  std
element                                                        
Mg             1.004654  1.007222  1.0  0.000950  0.001474  0.0
Al             1.003219  1.004997  1.0  0.000657  0.001020  0.0
Ca             1.002448  1.003821  1.0  0.000500  0.000780  0.0
Cl             1.003346  1.005193  1.0  0.000683  0.001060  0.0
Mn             1.003182  1.004941  1.0  0.000650  0.001008  0.0
Na             1.003659  1.005677  1.0  0.000747  0.001159  0.0
Ti             1.007128  1.011112  1.0  0.001455  0.002268  0.0
V              1.003570  1.005539  1.0  0.000729  0.001131  0.0
Sc             1.004542  1.007047  1.0  0.000927  0.001438  0.0
Tb             1.004565  1.007084  1.0  0.000932  0.001446  0.0
Sr             1.005816  1.009038  1.0  0.001187  0.001845  0.0
Co             1.003708  1.005752  1.0  0.000757  0.001174  0.0
Eu             1.003606  1.005594  1.0  0.000736  0.001142  0.0
Rb             1.004134  1.006413  1.0  0.000844  0.001309  0.0
Ba             1.005904  1.009175  1.0  0.001205  0.001873  0.0
Cs             1.004771  1.007405  1.0  0.000974  0.001511  0.0

# using periodic table

In [40]:
import neutron_analysis.shielding as shielding
import periodictable
import pandas as pd
import numpy as np

In [41]:
df_sigma = shielding.get_default_sigma_table()

In [43]:
df_sigma

,atomic_mass,sigma_a,sigma_s
element,,,
H,1.007940,0.332600,82.02
D,2.014102,0.000519,7.64
He,4.002602,0.007470,1.34
Li,6.941000,70.500000,1.37
Be,9.012182,0.007600,7.63
...,...,...,...
Ra,NaN,12.800000,13.00
Ac,NaN,NaN,NaN
Th,232.038100,7.370000,13.36


In [44]:
# check agains periodic table
df_check = (
    df_sigma
    .reset_index()
    .assign(
        atomic_mass2=lambda df: df['element'].apply(lambda x: getattr(periodictable, x).mass),
        sigma_a2=lambda df: df['element'].apply(lambda x: getattr(periodictable, x).neutron.absorption),
        sigma_s2=lambda df: df['element'].apply(lambda x: getattr(periodictable, x).neutron.total),
    )
    .assign(
        dmass=lambda x: np.abs(x['atomic_mass'] - x['atomic_mass2']),
        da=lambda x: np.abs(x['sigma_a'] - x['sigma_a2']),
        ds=lambda x: np.abs(x['sigma_s'] - x['sigma_s2']),
    )
    .set_index('element')
    
)

In [46]:
df_check[['atomic_mass','atomic_mass2','dmass']].dropna().query('dmass > 0')

,atomic_mass,atomic_mass2,dmass
element,,,
D,2.014102,2.014102,2.000000e-09
C,12.011000,12.010700,3.000000e-04
N,14.006740,14.006700,4.000000e-05
Na,22.989768,22.989770,2.000000e-06
Al,26.981539,26.981538,1.000000e-06
P,30.973762,30.973761,1.000000e-06
S,32.066000,32.065000,1.000000e-03
Cl,35.452700,35.453000,3.000000e-04
Ti,47.880000,47.867000,1.300000e-02


In [47]:
df_check[['sigma_a','sigma_a2','da']].dropna().query('da > 0')

,sigma_a,sigma_a2,da
element,,,
Nd,51.0,50.5,0.5


In [48]:
df_check[['sigma_s','sigma_s2','ds']].dropna().query('ds > 0')

,sigma_s,sigma_s2,ds
element,,,
Sm,39.0,39.400,0.400
U,8.9,8.908,0.008


In [249]:
reload(utils)

<module 'neutron_analysis.utils' from '/Users/wpk/Documents/Research/CurrentResearch/Projects/CCI/collaboration_646/Nick_sharp_workflow/neutron_analysis/neutron_analysis/utils.py'>

In [244]:
list(utils.reorder_names([...,'a'], list('abcd')))

['b', 'c', 'd', 'a']

In [238]:
def reorder_names(dims_supplied, dims_all):
    """
    Resolves a supplied list containing an ellispsis representing other items, to
    a generator with the 'realized' list of all items
    """
    if ... in dims_supplied:
        if len(set(dims_all)) != len(dims_all):
            raise ValueError("Cannot use ellipsis with repeated dims")
        if len([d for d in dims_supplied if d == ...]) > 1:
            raise ValueError("More than one ellipsis supplied")
        other_dims = [d for d in dims_all if d not in dims_supplied]
        for d in dims_supplied:
            if d == ...:
                yield from other_dims
            else:
                yield d
    else:
        if set(dims_supplied) ^ set(dims_all):
            raise ValueError(
                f"{dims_supplied} must be a permuted list of {dims_all}, unless `...` is included"
            )
        yield from dims_supplied



In [239]:
list(reorder_names(('d',...,'a'), list('abcd') ))

['d', 'b', 'c', 'a']